In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from google.colab import drive
drive.mount("/content/gdrive/")

Mounted at /content/gdrive/


In [3]:
import sys
sys.path.append("/content/gdrive/My Drive/nlu-project")

In [4]:
from corpus import MovieReviewsCorpus, MovieReviewsCorpusPhrases
from models import BiLSTM, BiLSTMAttention
from preprocess import MRPipelineTokens, MRPipelinePhrases
from utils import *
from torchtext.vocab import GloVe
from torchtext.vocab import FastText
from data import MovieReviewsDataset, get_data
import nltk

In [5]:
nltk.download("punkt")
nltk.download("movie_reviews")
nltk.download("subjectivity")
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package movie_reviews to /root/nltk_data...
[nltk_data]   Unzipping corpora/movie_reviews.zip.
[nltk_data] Downloading package subjectivity to /root/nltk_data...
[nltk_data]   Unzipping corpora/subjectivity.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [6]:
global_vectors = GloVe(name='840B', dim=300, cache = "/content/gdrive/My Drive/nlu-project/.vector_cache")

In [7]:
mr_pipeline = MRPipelineTokens()
corpus = MovieReviewsCorpus(mr_pipeline)

In [8]:
oov = check_coverage(corpus.vocab, global_vectors)

100%|██████████| 39275/39275 [00:01<00:00, 22466.75it/s]



Found embeddings for 92.58% of vocab
Found embeddings for  95.28% of all text


In [9]:
embedding_matrix = corpus.get_embedding_matrix(global_vectors, 300)
ds = corpus.get_indexed_corpus()

In [10]:
dataset = MovieReviewsDataset(ds)
train_loader, test_loader = get_data(128, dataset, collate_fn=collate)

/content/gdrive/My Drive/nlu-project/data.py:12: FutureWarning: The input object of type 'Tensor' is an array-like implementing one of the corresponding protocols (`__array__`, `__array_interface__` or `__array_struct__`); but not a sequence (or 0-D). In the future, this object will be coerced as if it was first converted using `np.array(obj)`. To retain the old behaviour, you have to either modify the type 'Tensor', or assign to an empty array created with `np.empty(correct_shape, dtype=object)`.
  self.corpus = np.array(raw_dataset[0], dtype = object)


In [11]:
def training_step(net, data_loader, optimizer, cost_function, device = 'cuda'):
  cumulative_loss = 0
  cumulative_accuracy = 0
  samples = 0

  net.train()

  for batch_idx, (inputs, targets) in enumerate(data_loader):

    inputs = inputs.to(device)
    targets = targets.to(device)
    in_size = targets.size(dim=0)

    outputs = net(inputs)

    loss = cost_function(outputs, targets)

    loss.backward()

    optimizer.step()

    optimizer.zero_grad()
    
    samples += in_size
    cumulative_loss += loss.item()
    _, predicted = outputs.max(dim=1)

    cumulative_accuracy += predicted.eq(targets).sum().item()

  return cumulative_loss/samples, (cumulative_accuracy/samples)*100

In [12]:
def test_step(net, data_loader, cost_function, device = 'cuda'):
  cumulative_loss = 0
  cumulative_accuracy = 0
  samples = 0

  net.eval()

  with torch.no_grad():

    for batch_idx, (inputs, targets) in enumerate(data_loader):
      inputs = inputs.to(device)
      targets = targets.to(device)
      in_size = targets.size(dim=0)

      outputs = net(inputs)

      loss = cost_function(outputs, targets)

      samples += in_size
      cumulative_loss += loss.item()
      _, predicted = outputs.max(dim=1)

      cumulative_accuracy += predicted.eq(targets).sum().item()

    return cumulative_loss/samples, (cumulative_accuracy/samples)*100


In [13]:
from torch.utils.data import DataLoader
from torch.optim import Adam
import torch.nn as nn

def main(train_loader, test_loader, embedding_matrix, device = "cuda", epochs = 10):

  net = BiLSTMAttention(embedding_matrix, device = device, input_size=300).to(device)

  optimizer = Adam(net.parameters(), 0.001, betas = (0.9, 0.999), amsgrad=True)

  cost_function = nn.CrossEntropyLoss()

  for e in range(epochs):
    print(f"epoch {e}:")
    train_loss, train_accuracy = training_step(net, train_loader, optimizer, cost_function, device)
    print(f"Training loss: {train_loss} \n Training accuracy: {train_accuracy}")
    test_loss, test_accuracy = test_step(net, test_loader, cost_function, device)
    print(f"Test loss: {test_loss} \n Test accuracy: {test_accuracy}")
    print("------------------------------------------------------------------")
  
  _, test_accuracy = test_step(net, test_loader, cost_function, device)


  return test_accuracy


In [16]:
!pip3 install torch torchtext torchvision torchaudio --extra-index-url https://download.pytorch.org/whl/cu116

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/, https://download.pytorch.org/whl/cu116


In [17]:
main(train_loader, test_loader, embedding_matrix, epochs=30)

epoch 0:


/content/gdrive/My Drive/nlu-project/utils.py:40: FutureWarning: The input object of type 'Tensor' is an array-like implementing one of the corresponding protocols (`__array__`, `__array_interface__` or `__array_struct__`); but not a sequence (or 0-D). In the future, this object will be coerced as if it was first converted using `np.array(obj)`. To retain the old behaviour, you have to either modify the type 'Tensor', or assign to an empty array created with `np.empty(correct_shape, dtype=object)`.
  X = np.array(list(X))
/content/gdrive/My Drive/nlu-project/utils.py:40: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  X = np.array(list(X))


Training loss: 0.004780624248087406 
 Training accuracy: 67.625
Test loss: 0.006847285926342011 
 Test accuracy: 64.25
------------------------------------------------------------------
epoch 1:
Training loss: 0.003231080211699009 
 Training accuracy: 82.375
Test loss: 0.006645809859037399 
 Test accuracy: 82.25
------------------------------------------------------------------
epoch 2:
Training loss: 0.0018964504916220903 
 Training accuracy: 91.1875
Test loss: 0.005972939282655716 
 Test accuracy: 85.75
------------------------------------------------------------------
epoch 3:
Training loss: 0.0009313559159636498 
 Training accuracy: 95.6875
Test loss: 0.004490049406886101 
 Test accuracy: 86.25
------------------------------------------------------------------
epoch 4:
Training loss: 0.0003248953731963411 
 Training accuracy: 99.3125
Test loss: 0.005463392287492752 
 Test accuracy: 78.25
------------------------------------------------------------------
epoch 5:
Training loss: 0.00

86.5